# 该程序是用来将Vaihingen在Val数据集的预测结果可视化并保存的

In [46]:
# 导入所需要用到的包
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision
import time
import torch.nn.functional as F
from numpy import *
from matplotlib import pyplot as plt
from PIL import Image
import d2l
import os
import numpy as np
from measure import SegmentationMetric
from dataset import img_dataset   # from xx.py文件 import里面的函数


from model_file.MACUNet import MACUNet      #MACUNet模型
from model_file.MANet import MANet         #MACUNet模型
from model_file.DRCANet import DRCANet      #DRCANet模型
from model_file.UNet import UNet          #UNet模型   
from model_file.ICNet.model import ICNet     #ICNet模型 
from model_file.BiSeNet.model import BiSeNet  #BiseNet模型 双边网络

from early_stopping import EarlyStopping
from tqdm import tqdm, trange
import glob

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
torch.backends.cudnn.enabled =True
# torch.backends.cudnn.benchmark = True
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

%config InlineBackend.figure_format = 'svg'

In [47]:
print(torch.cuda.device_count())

2


In [48]:
#一些参数的设置
batch_size=1
class_num = 6   #类别数 背景-1、建筑-2、道路-3、水-4、贫瘠-5、森林-6、农业-7。没有数据的区域被指定为0，应该忽略 
cuda = True
num_workers = 12 #读入数据时候的线程
band = 3       #通道数，RGB则为3,遥感图像还有其他多光谱的
# net = DRCANet(class_num)   #MACUNET网络##########################################################修改网络名字


# ###############Vaihingen数据####################
# train_path = '/root/master/ATL/dataset/Vaihingen/Vaihingen_512/Train' #训练集地址
# val_path = '/root/master/ATL/dataset/Vaihingen/Vaihingen_512/Val'     #验证集地址

# ###############Potsdam数据####################
train_path = '/root/master/ATL/dataset/Potsdam_512/Train' #训练集地址
val_path = '/root/master/ATL/dataset/Potsdam_512/Val'     #验证集地址


num_GPU = 1  #GPU的个数

# torch.cuda.set_device(0) #设置设备为GPU0？？？？？如果是多个GPU呢？？？
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [49]:
##########数据增强的方法，会直接应用到Dataset的 __getitem__(),Dataloader是通过调用__getitem__()来返回数据集的！
val_transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),#先除以255归一化
                                                torchvision.transforms.Normalize(mean=[.485, .456, .406],std=[.229, .224, .225])])

In [50]:
# ################加载训练集和验证集############
data_val = img_dataset(val_path, val_transform)
data_loader_val = torch.utils.data.DataLoader(dataset=data_val,
                                                # 这里注意batch size要对应放大倍数
                                                batch_size = 1, 
                                                shuffle = True,num_workers=12)
real_imgs = img_dataset(val_path)

print('-----------------------------------------')    
if data_val.__len__()!= 0:
    print(f'  -Val Dataset load successfully! total {data_val.__len__()}')
    print('  -Val Dataset images shape',data_val.__getitem__(0)[0].shape)
    print('  -Val Dataset label shape',data_val.__getitem__(0)[1].shape)
else:
    print('  --Val dataset load fail--')
print('-----------------------------------------')    

-----------------------------------------
  -Val Dataset load successfully! total 363
  -Val Dataset images shape torch.Size([3, 512, 512])
  -Val Dataset label shape (512, 512)
-----------------------------------------


In [51]:
# 加载模型并进行预测！！
########################################## 可视化预测值
GID_COLOR_MAP = dict(
    Background=(255, 0, 0),#0-背景-红色
    Building=(0, 0, 255), #1-建筑-蓝色
    di_ai_zhi_bei=(0, 255, 255), #2-低矮植被-青色
    Tree=(0, 255, 0), #3-树-绿色
    car=(255, 255, 0),#4-汽车-黄色
    bu_tou_shui_mian=(255, 255, 255), #5-不透水面-白色
)

def render(mask_image):
    new_mask = mask_image.astype(np.uint8) #把原来的图片读出来，存成uint8的格式
    cm = np.array(list(GID_COLOR_MAP.values())).astype(np.uint8) #取出COLOR_MAP中的值，存到列表
    color_img = cm[new_mask]  #这里就变成1024*1024*3的了？ why？？
    color_img = Image.fromarray(np.uint8(color_img)) #从np的array格式转换成PIL格式
    return color_img

In [64]:
###################加载模型，加载参数########################
#unet
# model_path = '/root/master/ATL/code/MACUNet/checkpoint/Potsdam/unet/05-08-15_42_24/model_weights.pth'
# save_file_path = '/root/master/ATL/code/MACUNet/结果图/Potsdam/unet/Size_512/'
# net_predict = UNet(3,class_num)

#drcanet
# model_path = '/root/master/ATL/code/MACUNet/checkpoint/Potsdam/drcanet/05-08-01_06_12/model_weights.pth'
# save_file_path = '/root/master/ATL/code/MACUNet/结果图/Potsdam/drcanet/Size_512/'
# net_predict = DRCANet(class_num) 

#manet
# model_path = '/root/master/ATL/code/MACUNet/checkpoint/Potsdam/manet/05-08-01_04_08/model_weights.pth'
# save_file_path = '/root/master/ATL/code/MACUNet/结果图/Potsdam/manet/Size_512/'
# net_predict = MANet(band,class_num) 

#macunet
# model_path = '/root/master/ATL/code/MACUNet/checkpoint/Potsdam/macunet/05-08-01_03_23/model_weights.pth'
# save_file_path = '/root/master/ATL/code/MACUNet/结果图/Potsdam/macunet/Size_512/'
# net_predict = MACUNet(band,class_num) 

# icnet
# model_path = '/root/master/ATL/code/MACUNet/checkpoint/Potsdam/icnet/05-09-12_05_44/model_weights.pth'
# save_file_path = '/root/master/ATL/code/MACUNet/结果图/Potsdam/icnet/Size_512/'
# net_predict = ICNet(nclass=class_num)

#bisenet
model_path = '/root/master/ATL/code/MACUNet/checkpoint/Potsdam/bisenet/05-09-10_07_24/model_weights.pth'
save_file_path = '/root/master/ATL/code/MACUNet/结果图/Potsdam/bisenet/Size_512/'
net_predict = BiSeNet(num_classes=class_num, context_path='resnet18')

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
net_predict = nn.DataParallel(net_predict);   #多gpu训练的模型，要加这一句才行
net_predict.load_state_dict(torch.load(model_path)) #如果在服务器，不需要map_location='cuda:0'
net_predict.cuda();

In [65]:
################读取一张需要进行预测的图片######################
try:
    os.makedirs(save_file_path) #生成一个outfile的文件夹
except OSError:
    pass

In [66]:
start = time.time()    
for i in trange(50,desc='保存ing...'):
    real_img = real_imgs.__getitem__(i)[0]
    
    X = data_val.__getitem__(i)[0]  #更换要读取的图片 ######  real img的地址
    
    X = X.unsqueeze(0) #变成四维的张量
    Y = net_predict(X.cuda()).argmax(dim=1) #X.cuda()#
    Y.shape
    Y = Y.reshape(Y.shape[1], Y.shape[2]) #变成(1024,1024)
    Y.shape
    predict_images= Y.cpu().numpy()
    
    

    ########################################## 可视化预测值
    predict_img = render(predict_images)

    ############################################ 读取vis_png 进行比较
    vis_png_path = val_path + '/vis/'+data_val.images_list[i]
    guound_truth_img = np.array(Image.open(vis_png_path))

    #################################################画图并保存文件
    plt.subplot(1,3,1),plt.imshow(real_img),plt.title('real img'),plt.xticks([]),plt.yticks([]) #隐藏掉坐标轴
    plt.subplot(1,3,2),plt.imshow(guound_truth_img),plt.title('ground truth'),plt.xticks([]),plt.yticks([])
    plt.subplot(1,3,3),plt.imshow(predict_img),plt.title('predict img'),plt.xticks([]),plt.yticks([])

    f = plt.gcf()
    save_path = os.path.join(save_file_path,data_val.images_list[i])
    f.savefig(save_path,dpi=500,bbox_inches = 'tight')
    plt.clf()  #清理内存，不然会越来越慢 越来越慢
    
#     del X, predict_img, predict_images, Y ,f
#     torch.cuda.empty_cache()

end = time.time()
print(f'共保存完成 {data_val.__len__()} 张图像, 耗时 {end-start} 秒')

保存ing...: 100%|██████████| 50/50 [00:33<00:00,  1.48it/s]

共保存完成 363 张图像, 耗时 33.87263035774231 秒


<Figure size 432x288 with 0 Axes>